## Setup
### Imports

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import sys
import os
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

IS_COLAB = 'google.colab' in sys.modules
if IS_COLAB:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    
import tensorflow as tf
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from datetime import datetime
from pprint import pprint
import time
import matplotlib.pyplot as plt

%matplotlib notebook

### Setup GPU

In [ ]:
# Set GPU memory growth
# Allows to only as much GPU memory as needed
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

### Constants

In [ ]:
SEED = 1234
tf.random.set_seed(SEED)

if IS_COLAB:
    DATASET_PATH = '/content/drive/My Drive'
else:
    DATASET_PATH = '/kaggle/input/ann-and-dl-image-segmentation'

CWD = os.getcwd()
DATASET_DIR_NAME = 'Segmentation_Dataset'
EXPERIMENTS_PATH = os.path.join(CWD, 'segmentation_experiments')

EPOCHS = 150
LEARNING_RATE = 1e-3
SHOW_PLOTS = False
USE_DATA_AUGMENTATION = True
VALIDATION_SPLIT = 0.2
BATCH_SIZE = 4
IMAGE_WIDTH = 256
IMAGE_HEIGHT = 256
OUTPUT_CHANNELS = 1
RGB_CHANNELS = 3
NUMBER_OF_CLASSES = 1 # house / not-house
EARLY_STOP = True
SAVE_PREDICTIONS = True

print("cwd: %s" % CWD)

## Data Generators

In [ ]:
if USE_DATA_AUGMENTATION:
    train_img_data_gen = ImageDataGenerator(rotation_range=10,
                                            width_shift_range=10,
                                            height_shift_range=10,
                                            zoom_range=0.3,
                                            horizontal_flip=True,
                                            vertical_flip=True,
                                            fill_mode='constant',
                                            cval=0,
                                            rescale=1. / 255,
                                            validation_split=VALIDATION_SPLIT)

    train_mask_data_gen = ImageDataGenerator(rotation_range=10,
                                             width_shift_range=10,
                                             height_shift_range=10,
                                             zoom_range=0.3,
                                             horizontal_flip=True,
                                             vertical_flip=True,
                                             fill_mode='constant',
                                             cval=0,
                                             rescale=1. / 255,
                                             validation_split=VALIDATION_SPLIT)
else:
    train_img_data_gen = ImageDataGenerator(rescale=1. / 255, validation_split=VALIDATION_SPLIT)
    train_mask_data_gen = ImageDataGenerator(rescale=1. / 255, validation_split=VALIDATION_SPLIT)

test_img_data_gen = ImageDataGenerator(rescale=1./255)

# Create generators to read images from dataset directory
# -------------------------------------------------------

# path to dataset
dataset_dir = os.path.join(DATASET_PATH, DATASET_DIR_NAME)
# path to train dataset
train_dir = os.path.join(dataset_dir, 'training')
# path to test dataset
test_dir = os.path.join(dataset_dir, 'test')

print('dataset %s \ntrain dir %s \ntest dir %s \n' % (dataset_dir, train_dir, test_dir))

### Dataset generation

In [ ]:
## Training images
train_img_gen = train_img_data_gen.flow_from_directory(
    os.path.join(train_dir, 'images'),
    subset='training',  # subset of data
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
#     color_mode='grayscale',
    class_mode=None,
    shuffle=True,
    interpolation='bilinear',
    seed=SEED)
## Training masks
train_mask_gen = train_mask_data_gen.flow_from_directory(
    os.path.join(train_dir, 'masks'),
    subset='training',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode=None,
    shuffle=True,
    interpolation='bilinear',
    seed=SEED)

## Validation images
valid_img_gen = train_img_data_gen.flow_from_directory(
    os.path.join(train_dir, 'images'),
    subset='validation',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
#     color_mode='grayscale',
    class_mode=None,
    shuffle=False,
    interpolation='bilinear',
    seed=SEED)

## Validation masks
valid_mask_gen = train_mask_data_gen.flow_from_directory(
    os.path.join(train_dir, 'masks'),
    subset='validation',
    target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode=None,
    shuffle=False,
    interpolation='bilinear',
    seed=SEED)

## Test images
test_img_gen = test_img_data_gen.flow_from_directory(os.path.join(test_dir, 'images'),
                                                     target_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                                     batch_size=BATCH_SIZE, 
                                                     class_mode=None, # Because we have no class subfolders in this case
                                                     shuffle=False,
                                                     interpolation='bilinear',
                                                     seed=SEED)

## Validation generator
valid_gen = zip(valid_img_gen, valid_mask_gen)
## Training generator
train_gen = zip(train_img_gen, train_mask_gen)
## Test generator ==> since we dont have masks..
test_gen = test_img_gen

### Create dataset objects

In [ ]:
def prepare_target(x_, y_):
    y_ = tf.cast(tf.expand_dims(y_[..., 0], -1), tf.int32)
    return x_, tf.where(y_ > 0, y_ - 1, y_ + 1)

train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS], [None, IMAGE_HEIGHT, IMAGE_WIDTH, OUTPUT_CHANNELS]))
train_dataset = train_dataset.map(prepare_target)
# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS], [None, IMAGE_HEIGHT, IMAGE_WIDTH, OUTPUT_CHANNELS]))
valid_dataset = valid_dataset.map(prepare_target)
# Repeat
valid_dataset = valid_dataset.repeat()

# Test
# ----------
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS], [None, IMAGE_HEIGHT, IMAGE_WIDTH, OUTPUT_CHANNELS]))
test_dataset = test_dataset.map(prepare_target)
# Repeat
test_dataset = test_dataset.repeat()

#### Test data generator

In [ ]:
if SHOW_PLOTS:
    fig, ax = plt.subplots(1, 2)
    fig.show()

    # Assign a color to each class
    colors_dict = {}
    colors_dict[0] = [255, 255, 255]  # foreground
    colors_dict[1] = [0, 0, 0]  # background
    colors_dict[2] = [3, 82, 252] # contours

    iterator = iter(train_dataset)

    for _ in range(2):
        augmented_img, target = next(iterator)
        augmented_img = augmented_img[0]   # First element
        augmented_img = augmented_img * 255  # denormalize

        target = np.array(target[0, ..., 0])   # First element (squeezing channel dimension)

        # Assign colors (just for visualization)
        target_img = np.zeros([target.shape[0], target.shape[1], 3])

        target_img[np.where(target == 0)] = colors_dict[0]
        target_img[np.where(target == 1)] = colors_dict[1]
        target_img[np.where(target == 2)] = colors_dict[2]

        ax[0].imshow(np.uint8(augmented_img))
        ax[1].imshow(np.uint8(target_img))

        fig.canvas.draw()
        time.sleep(1)

## Convolutional Neural Network
### Functions

In [ ]:
# IoU metric function
def iou_metric(y_true, y_pred):
    # from pobability to predicted class {0, 1}
    y_pred = tf.cast(y_pred > 0.5, tf.float32) # when using sigmoid. Use argmax for softmax
    # A and B
    intersection = tf.reduce_sum(y_true * y_pred)
    # A or B
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    # IoU
    return intersection / union

# Create a model 
def create_model(depth, start_f, num_classes, dynamic_input_shape):
    model = tf.keras.Sequential()

    # Encoder
    # -------
    for i in range(depth):
        if i == 0:
            if dynamic_input_shape:
                input_shape = [None, None, RGB_CHANNELS]
            else:
                input_shape = [IMAGE_HEIGHT, IMAGE_WIDTH, RGB_CHANNELS]
        else:
            input_shape = [None]

        model.add(
            tf.keras.layers.Conv2D(filters=start_f,
                                   kernel_size=(3, 3),
                                   strides=(1, 1),
                                   padding='same',
                                   input_shape=input_shape))
        model.add(tf.keras.layers.ReLU())
        model.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

        start_f *= 2

    # Decoder
    # -------
    for i in range(depth):
        model.add(tf.keras.layers.UpSampling2D(2, interpolation='bilinear'))
        model.add(tf.keras.layers.Conv2D(filters=start_f // 2,
                                   kernel_size=(3, 3),
                                   strides=(1, 1),
                                   padding='same'))

        model.add(tf.keras.layers.ReLU())

        start_f = start_f // 2

    # Prediction Layer
    # ----------------
    model.add(
        tf.keras.layers.Conv2D(filters=num_classes,
                               kernel_size=(1, 1),
                               strides=(1, 1),
                               padding='same',
                               activation='sigmoid'))

    return model

### Create the model

In [ ]:
model = create_model(depth=4,
                     start_f=4,
                     num_classes=NUMBER_OF_CLASSES,
                     dynamic_input_shape=False)

# Visualize created model as a table
model.summary()

# Visualize initialized weights
# print(model.weights)

### Compile the model
#### Functions

In [ ]:
def dice_coeficcient(y_true, y_pred, smooth=1):
    from keras import backend as K
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1) + smooth)

def dice_loss(y_true, y_pred):
    return 1-dice_coeficcient(y_true, y_pred)

In [ ]:
# Loss
# Sparse Categorical Crossentropy to use integers (mask) instead of one-hot encoded labels
# loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
loss = 'binary_crossentropy'

# optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)

# Validation metrics
metrics = [iou_metric, 'accuracy']

# Compile Model
model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

## Training

In [ ]:
callbacks = []

if not os.path.exists(EXPERIMENTS_PATH):
    os.makedirs(EXPERIMENTS_PATH)

now = datetime.now().strftime('%b%d_%H-%M-%S')

model_name = 'CNN'

exp_dir = os.path.join(EXPERIMENTS_PATH, model_name + '_' + str(now))
if not os.path.exists(exp_dir):
    os.makedirs(exp_dir)

# Model checkpoint
# ----------------
ckpt_dir = os.path.join(exp_dir, 'ckpts')
if not os.path.exists(ckpt_dir):
    os.makedirs(ckpt_dir)

ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(ckpt_dir, 'cp_{epoch:02d}.ckpt'), 
                                                   save_weights_only=False)  # False to save the model directly
callbacks.append(ckpt_callback)

# Visualize Learning on Tensorboard
# ---------------------------------
# tb_dir = os.path.join(exp_dir, 'tb_logs')
# if not os.path.exists(tb_dir):
#     os.makedirs(tb_dir)
    
# # By default shows losses and metrics for both training and validation
# tb_callback = tf.keras.callbacks.TensorBoard(log_dir=tb_dir,
#                                              profile_batch=0,
#                                              histogram_freq=0)  # if 1 shows weights histograms
# callbacks.append(tb_callback)

# Early Stopping
# --------------
if EARLY_STOP:
    es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)
    callbacks.append(es_callback)


model.fit(x=train_dataset, epochs=EPOCHS, 
                          steps_per_epoch = (len(train_img_gen)//BATCH_SIZE),
                          validation_data=valid_gen, 
                          validation_steps=(len(valid_img_gen)//BATCH_SIZE), 
                          callbacks=callbacks)


## Evaluation

In [ ]:
evaluation_output = model.evaluate(x=valid_dataset, steps=len(valid_img_gen), verbose=0)

print('Loss: %s \nIoU Metric: %s \nAccuracy: %s' % (evaluation_output[0], evaluation_output[1], evaluation_output[2]))

## Final result
### Functions

In [ ]:
# Compute predictions
def generate_predictions(model):
    # Cycle over test images
    test_img_dir = os.path.join(test_dir, 'images', 'img')

    # s[:10] predict until 10th image
    image_filenames = next(os.walk(test_img_dir))[2]

    results = {}

    for filename in image_filenames:
        # test images are in RGB, hence no need to transform them.
        img = Image.open(os.path.join(test_img_dir, filename))
        img = img.resize((IMAGE_HEIGHT, IMAGE_WIDTH))  # target size

        # data_normalization
        img_array = np.array(img)  #
        # img_array = img_array * 1. / 255  # normalization
        img_array = np.expand_dims(img_array, axis=0) # to fix dims of input in the model

        print("prediction for {}...".format(filename))
        predictions = model.predict(img_array)

        # Get predicted class as the index corresponding to the maximum value in the vector probability
        predicted_mask = tf.argmax(predictions, axis=-1)
        predicted_mask = predicted_mask[0]
        target = np.array(predicted_mask)
        target = rle_encode(target)
        
        # print(target.shape)
        results[filename[:-4]] = target

    # create_csv(results)
    print('Num. of labeled images', results.__len__())
    return results
    
## Create CSV file
def create_csv(results, results_dir='./'):
    csv_fname = 'results_'
    csv_fname += datetime.now().strftime('%b%d_%H-%M-%S') + '.csv'
    csv_fname = os.path.join(results_dir, csv_fname)
    with open(csv_fname, 'w') as f:
        f.write('ImageId,EncodedPixels,Width,Height\n')
        for key, value in results.items():
            f.write(key + ',' + str(value) + ',' + '256' + ',' + '256' + '\n')
        
def rle_encode(img):
    # Flatten column-wise
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)
        

### Generate predictions

In [ ]:
predictions = generate_predictions(model)

#### Save predictions

In [ ]:
if SAVE_PREDICTIONS:
    predictions_dir = os.path.join(CWD, 'predictions')
    if not os.path.exists(predictions_dir):
        os.makedirs(predictions_dir)
    create_csv(predictions, predictions_dir)